In [ ]:
# Nanopore pipeline to basecall and demultiple nanopore fast5 raw data

In [ ]:
# print work directory
pwd

In [ ]:
# change directory
cd /home/andhika/extrastorage/MinION/dum_fishy/20190313_0937_MN31442_FAK11288_e5927bdc/

In [ ]:
# basecalling using guppy

~/extrastorage/SOFTWARE/ont_guppy_cpu/bin/guppy_basecaller \
--input_path ~/extrastorage/MinION/dum_fishy/20190313_0937_MN31442_FAK11288_e5927bdc/fast5 \
--save_path ~/extrastorage/MinION/dum_fishy/20190313_0937_MN31442_FAK11288_e5927bdc/basecall_2 \
--flowcell FLO-MIN106 --kit SQK-LSK109

In [ ]:
# combine all barcode into 1 file
cat basecall/*fastq > combined.fastq

In [ ]:
# check
grep -c '^+$' combined.fastq
# 1,800,063 million reads

In [ ]:
# demultiplex basecalled fast5 reads
qcat -f combined.fastq -b final_NPreads &> final_log

In [ ]:
# open file
less final_log 

# quit
q

In [ ]:
## EXTRACT BY SAMPLE
# create folder for analyzing by sample
mkdir dhika_files
cd dhika_files

In [ ]:
# create folder for barcode sample
mkdir sample

In [ ]:
# Tried to extract by barcode basecall to barcode
cat basecall/*.fastq | qcat -b dhika_files/sample/

In [ ]:
# create folder to converst fastq to fasta
mkdir sample_fasta

In [ ]:
# convert fastq to fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode01.fastq > sample_fasta/barcode01.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode02.fastq > sample_fasta/barcode02.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode03.fastq > sample_fasta/barcode03.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode04.fastq > sample_fasta/barcode04.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode05.fastq > sample_fasta/barcode05.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode06.fastq > sample_fasta/barcode06.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode07.fastq > sample_fasta/barcode07.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode08.fastq > sample_fasta/barcode08.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode09.fastq > sample_fasta/barcode09.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode10.fastq > sample_fasta/barcode010.fasta

In [ ]:
# Assembling
# We use genomeSize=16k as it is the expected size of a mitochondrial genome.
# Only 3 barcodes successfully assembled (barcode 2, barcode 3 and barcode 5)
canu -d assemble2/ -p barcode2 genomeSize=16k minReadLength=500 -nanopore-raw sample/barcode02.fastq 2> canu2.log
canu -d assemble3/ -p barcode3 genomeSize=16k minReadLength=500 -nanopore-raw sample/barcode03.fastq 2> canu3.log
canu -d assemble5/ -p barcode5 genomeSize=16k minReadLength=500 -nanopore-raw sample/barcode05.fastq 2> canu5.log

In [ ]:
# Organized the contigs assemblages
mkdir assemble_fasta
cp assemble2/barcode2.contigs.fasta assemble_fasta/
cp assemble3/barcode3.contigs.fasta assemble_fasta/
cp assemble5/barcode5.contigs.fasta assemble_fasta/

In [ ]:
# We then blast the contigs agains nt
blastn -db /extrastorage/home/refdbs/blastdb/nt/nt -query assemble_fasta/barcode2.contigs.fasta \
-out blastn/barcode2_contigs_nt.blastn -outfmt \
"6 qseqid sseqid pident length evalue salltitles sallacc sstart send qstart qend" -max_target_seqs 2  \
-max_hsps 2 -num_threads 23 -culling_limit 1 -evalue 0.001

blastn -db /extrastorage/home/refdbs/blastdb/nt/nt -query assemble_fasta/barcode3.contigs.fasta \
-out blastn/barcode3_contigs_nt.blastn -outfmt \
"6 qseqid sseqid pident length evalue salltitles sallacc sstart send qstart qend" -max_target_seqs 2  \
-max_hsps 2 -num_threads 23 -culling_limit 1 -evalue 0.001

blastn -db /extrastorage/home/refdbs/blastdb/nt/nt -query assemble_fasta/barcode5.contigs.fasta \
-out blastn/barcode5_contigs_nt.blastn -outfmt \
"6 qseqid sseqid pident length evalue salltitles sallacc sstart send qstart qend" -max_target_seqs 2  \
-max_hsps 2 -num_threads 23 -culling_limit 1 -evalue 0.001

In [ ]:
# open the blastn files

In [ ]:
cat blastn/barcode2_contigs_nt.blastn

## Barcode 2 
#tig00000001	gi|442564744|dbj|AB731722.1|	98.462	65	1.17e-20	Prionace glauca DNA, microsatellite:FV6T5\
#AB731722	1	65	1139	1203

In [ ]:
cat blastn/barcode3_contigs_nt.blastn

## Barcode 3
#tig00000178	gi|442564744|dbj|AB731722.1|	98.462	65	1.08e-20	Prionace glauca DNA, microsatellite:FV6T5	\
    AB731722	65	1	2686	2750

In [ ]:
cat blastn/barcode5_contigs_nt.blastn

## Barcode 5
#tig00000001	gi|19067986|gb|AY049833.1|	99.944	1780	0.0	Galeocerdo cuvier 18S ribosomal RNA gene, \
#complete sequence	AY049833	1	1780	664	2442

In [ ]:
# We count how many genes blast assings to each shark species

In [ ]:
cut -f6 blastn/barcode2_contigs_nt.blastn | sort | uniq -c | sort -nr

## Barcode 2
#3 Triakis scyllium IL-1 gene for interleukin-1beta, complete cds
#3 Ginglymostoma cirratum clone GC_Ba-557B6, complete sequence
#2 Triakis scyllium MIP3 gene for macrophage inflammatory protein-3 alpha, complete cds
#2 Sphyrna lewini GRLN gene for preproghrelin, complete cds
#2 Heterodontus francisci Evx2 (evx2), HoxD14 (HoxD14), HoxD13 (HoxD13), HoxD12 (HoxD12), HoxD11 (HoxD11), HoxD10 (HoxD10), HoxD9 (HoxD9), HoxD8 (HoxD8), HoxD5 (HoxD5), HoxD4 (HoxD4), HoxD3 (HoxD3), HoxD2 (HoxD2), and HoxD1 (HoxD1) genes, complete cds
#2 Carcharhinus plumbeus Ig lambda light chain gene, complete cds
#1 Urobatis halleri clone Uha111 microsatellite sequence
#1 Triakis scyllium TNFRSF6B gene for tumor necrosis factor receptor superfamily, member 6b, complete cds
#1 Scyliorhinus canicula Cluster_HOXB sequence
#1 Prionace glauca DNA, microsatellite:FV6T5
#1 Ginglymostoma cirratum beta-2 microglobulin (bm2) gene, complete sequence
#1 Chiloscyllium punctatum antithrombin (serpinc1) gene, complete cds
#1 Carcharhinus plumbeus recombination activating gene 1 (RAG1) gene, partial cds; and recombination activating gene 2 (RAG2) gene, complete cds

In [ ]:
cut -f6 blastn/barcode3_contigs_nt.blastn | sort | uniq -c | sort -nr

## Barcode 3
#3 Ginglymostoma cirratum clone GC_Ba-557B6, complete sequence
#2 Carcharhinus plumbeus recombination activating gene 1 (RAG1) gene, partial cds; and recombination activating gene 2 (RAG2) gene, complete cds
#1 Triakis scyllium MIP3 gene for macrophage inflammatory protein-3 alpha, complete cds
#1 Sphyrna lewini GRLN gene for preproghrelin, complete cds
#1 Scyliorhinus canicula Cluster_HOXD sequence
#1 Prionace glauca DNA, microsatellite:FV6T5
#1 Heterodontus francisci Evx2 (evx2), HoxD14 (HoxD14), HoxD13 (HoxD13), HoxD12 (HoxD12), HoxD11 (HoxD11), HoxD10 (HoxD10), HoxD9 (HoxD9), HoxD8 (HoxD8), HoxD5 (HoxD5), HoxD4 (HoxD4), HoxD3 (HoxD3), HoxD2 (HoxD2), and HoxD1 (HoxD1) genes, complete cds
#1 Gallus gallus fosmid J_AD-95M2, complete sequence
#1 Gallus gallus BAC clone CH261-185L12 from chromosome w, complete sequence
#1 Galeocerdo cuvier microsatellite TGR1185 sequence
#1 Carcharhinus sorrah microsatellite CS08 sequence
#1 Carcharhinus plumbeus Ig lambda light chain gene, complete cds
#1 Callorhinchus milii clone P21F19.Kidney.k16 mRNA sequence


In [ ]:
cut -f6 blastn/barcode5_contigs_nt.blastn | sort | uniq -c | sort -nr

## Barcode 5
#6 Sphyrna lewini GRLN gene for preproghrelin, complete cds
#4 Triakis scyllium MIP3 gene for macrophage inflammatory protein-3 alpha, complete cds
#4 Squalus acanthias cd79a gene for B-cell antigen receptor complex-associated protein alpha chain
#4 Scyliorhinus canicula Fam45a protein (Fam45a), Eif3a protein (Eif3a), and Nanos1A protein (Nanos1A) genes, complete cds
#4 PREDICTED: Rhincodon typus uncharacterized LOC109924674 (LOC109924674), ncRNA
#4 PREDICTED: Rhincodon typus uncharacterized LOC109924588 (LOC109924588), transcript variant X1, ncRNA
#4 Ginglymostoma cirratum clone GC_Ba-557B6, complete sequence
#4 Carcharhinus plumbeus recombination activating gene 1 (RAG1) gene, partial cds; and recombination activating gene 2 (RAG2) gene, complete cds
#3 Galeocerdo cuvier mitochondrion, complete genome
#2 Squalus acanthias clone SA_Bc-133E21, complete sequence
#2 Scyliorhinus canicula Cluster_HOXB sequence
#2 Hexanchus griseus microsatellite Hgri157 sequence
#2 Ginglymostoma cirratum beta-2 microglobulin (bm2) gene, complete sequence
#2 Galeocerdo cuvier microsatellite TGR1185 sequence
#2 Chiloscyllium punctatum antithrombin (serpinc1) gene, complete cds
#2 Carcharhinus plumbeus T cell receptor gamma (TCRG) gene, partial sequence
#1 Zebrafish DNA sequence from clone DKEY-204A24 in linkage group 19, complete sequence
#1 Urobatis halleri clone Uha111 microsatellite sequence
#1 Triakis scyllium TNFRSF6B gene for tumor necrosis factor receptor superfamily, member 6b, complete cds
#1 Triakis scyllium IL-1 gene for interleukin-1beta, complete cds
#1 Triakis scyllium gene for CC chemokine, complete cds
#1 Scyliorhinus canicula Cluster_HOXD sequence
#1 Scophthalmus maximus chromosome 4
#1 Prionace glauca DNA, microsatellite:DAE7L
#1 PREDICTED: Rhincodon typus zinc finger protein 345-like (LOC109925924), mRNA
#1 PREDICTED: Rhincodon typus zinc finger protein 271-like (LOC109912386), mRNA
#1 PREDICTED: Rhincodon typus zinc finger protein 239-like (LOC109912674), transcript variant X1, mRNA
#1 PREDICTED: Rhincodon typus uncharacterized LOC109930996 (LOC109930996), ncRNA
#1 PREDICTED: Rhincodon typus uncharacterized LOC109926405 (LOC109926405), ncRNA
#1 PREDICTED: Rhincodon typus uncharacterized LOC109925472 (LOC109925472), ncRNA
#1 PREDICTED: Rhincodon typus uncharacterized LOC109914972 (LOC109914972), ncRNA
#1 PREDICTED: Rhincodon typus uncharacterized LOC109913125 (LOC109913125), ncRNA
#1 PREDICTED: Rhincodon typus 5-aminolevulinate synthase, erythroid-specific, mitochondrial-like (LOC109917145), mRNA
#1 PREDICTED: Pelodiscus sinensis uncharacterized LOC106731724 (LOC106731724), mRNA
#1 PREDICTED: Callorhinchus milii type-2 angiotensin II receptor-like (LOC103189739), mRNA
#1 Oryzias latipes strain HNI chromosome 23
#1 Negaprion brevirostris microsatellite LS44 sequence
#1 Leucoraja erinacea Hoxa gene cluster, complete sequence
#1 Hexanchus griseus microsatellite Hgri1237 sequence
#1 Heterodontus francisci Evx2 (evx2), HoxD14 (HoxD14), HoxD13 (HoxD13), HoxD12 (HoxD12), HoxD11 (HoxD11), HoxD10 (HoxD10), HoxD9 (HoxD9), HoxD8 (HoxD8), HoxD5 (HoxD5), HoxD4 (HoxD4), HoxD3 (HoxD3), HoxD2 (HoxD2), and HoxD1 (HoxD1) genes, complete cds
#1 Ginglymostoma cirratum ZFP112, b2m, brd2 genes for similar to zinc finger protein 112, beta 2 microglobulin, bromodomain containing 2, complete cds
#1 Ginglymostoma cirratum proto-oncogene tyrosine-protein kinase LCK (LCK) mRNA, complete cds
#1 Galeocerdo cuvier voucher eFish-1155 mitochondrion, complete genome
#1 Galeocerdo cuvier internal transcribed spacer 2, and 28S ribosomal RNA gene, partial sequence
#1 Galeocerdo cuvier 18S ribosomal RNA gene, complete sequence
#1 Cyprinus carpio genome assembly common carp genome, scaffold: LG41, chromosome: 41